<a href="https://colab.research.google.com/github/HausReport/ClubRaiders/blob/master/ClubRaiders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

# Refreshing the data


In [30]:
import requests

#
# Gets the most recent data dumps from eddb.io
#
url = 'https://eddb.io/archive/v6/systems_populated.jsonl'
r = requests.get(url, allow_redirects=True)
open('data/systems_populated.jsonl', 'wb').write(r.content)

url = 'https://eddb.io/archive/v6/stations.jsonl'
r = requests.get(url, allow_redirects=True)
open('data/stations.jsonl', 'wb').write(r.content)

url = 'https://eddb.io/archive/v6/factions.jsonl'
r = requests.get(url, allow_redirects=True)
open('data/factions.jsonl', 'wb').write(r.content)


15842334

In [0]:
import pandas as pd
import json
import datetime
from named.Faction import Faction
from named.FactionInstance import FactionInstance
from named.Station import Station
from named.InhabitedSystem import InhabitedSystem

import Club

all_factions_dict: Faction = {}
# all_stations_dict : Station = {}
all_systems_dict: InhabitedSystem = {}
club_factions_dict: Faction = {}
club_systems_arr: FactionInstance = []

#
# Identified Pro-Club Faction Names
#


with open("data/factions.jsonl", 'r') as handle:
    for line in handle:
        lCurFaction = json.loads(line)
        lCurFactionId = int(lCurFaction[ 'id' ])
        curFaction = Faction(lCurFaction)
        all_factions_dict[ lCurFactionId ] = curFaction
        if Club.proClubFaction(curFaction):
            club_factions_dict[ lCurFactionId ] = curFaction

# with open(stations_file, 'r') as handle:
#      for line in handle:
#         station = json.loads(line)
#         system_id = station[ SYSTEM_ID ]
#         current_station = stations_dict.get(system_id)

with open("data/systems_populated.jsonl", 'r') as handle:
    for line in handle:
        sys = json.loads(line)
        tid = int(sys[ 'id' ])
        foo = InhabitedSystem(sys)
        all_systems_dict[ tid ] = foo


def desiredState(state_dict):
    for state in state_dict:
        if (state[ 'id' ] == 64): return 64
        if (state[ 'id' ] == 65): return 65
        if (state[ 'id' ] == 73): return 73
        if (state[ 'id' ] == 96): return 96
        if (state[ 'id' ] == 104): return 104

    return 0


for tsys in all_systems_dict.values():
    ttsys: InhabitedSystem = tsys
    mfp = ttsys.getMinorFactionPresences()
    for faction_ptr in mfp:
        if faction_ptr is None:
            continue
        faction_id = int(faction_ptr[ 'minor_faction_id' ])
        if faction_id is None:
            continue
        if faction_id in club_factions_dict:
            fac = club_factions_dict[ faction_id ]
            facname = fac.get_name2()
            if (facname.startswith("*")): continue  # filters player factions

            sysname = ttsys.get_name()
            facHome = fac.get_homesystem_id()
            sysid = ttsys.get_id()
            vulnerable = True
            if (sysid == facHome): vulnerable = False

            govt = ttsys.getGovernment()
            # allg = fac.get_allegiance()

            inf = faction_ptr[ 'influence' ]
            # sinf = '{:04.2f}'.format(inf)
            # print(sinf)

            # updated = ttsys.getUpdated();
            # date = datetime.datetime.utcfromtimestamp(updated)
            # ds = date.strftime("%m/%d/%Y %H:%M:%S")

            active_states = json.dumps(faction_ptr[ 'active_states' ])
            hasWar = desiredState(faction_ptr[ 'active_states' ])
            if (govt == "Anarchy"):
                hasWar = -16
            if (hasWar == 0 and inf <= 3.5 and inf > 0.0):
                hasWar = -15
            if (hasWar == 104 and inf > 10.0):
                hasWar = 0

            sysIns = FactionInstance(fac, ttsys, inf, hasWar)
            club_systems_arr.append(sysIns)
            #print(facname + "," + sysname)
            #print("=====================================================================================")
            # print(facname + "," + sysname + "," + x + "," + y + "," + z + "," + allg + "," + sinf + "," + war+ "," + ds )  # + "," + allg)



# New Section

In [76]:
from IPython.display import HTML, display
import tabulate

arr = []

for xcs in club_systems_arr:
    cs: FactionInstance = xcs
    vulnerable = cs.isVulnerable()
    if (vulnerable == False): continue
    arr.append(cs.asArray())

tabulate.tabulate(arr, ['Faction','System','x','y','x','Allegiance','Influence','Vuln.','Updated'],tablefmt='html')

Faction,System,x,y,x,Allegiance,Influence,Vuln.,Updated
Worster Insurance,64 Ceti,-37.06,-100.06,-79.12,Independent,2.9,Low Influence,20-May-2020 20:12
Hodack Prison Colony,Akandi,62.03,10.75,38.88,Independent,4.7,LowInf+Inf Fail,21-May-2020 00:55
Trumbo Emperor's Grace,Aowica,116.84,-55.84,84.47,Empire,12.69,Election,20-May-2020 22:30
Aracenus Emperor's Grace,Aracenus,111.31,-65.88,68,Empire,8.59,Civil War,20-May-2020 05:15
Asase Ya Emperor's Grace,Asase Ya,96.03,-126.31,21.44,Empire,8.18,Civil War,20-May-2020 14:53
Wo 9812 B Emperor's Grace,Bidjogo,-49.47,-137,51.56,Empire,7.6,War,19-May-2020 16:06
NLTT 56379 Emperor's Grace,Boloch,5.88,-105,45.5,Empire,8.29,Election,20-May-2020 20:28
Kitche Emperor's Grace,Boloch,5.88,-105,45.5,Empire,8.29,Election,20-May-2020 20:28
Botaiti Emperor's Grace,Botaiti,111.03,-143.69,-6.53,Empire,13.19,Election,16-May-2020 22:52
Sirius Hyperspace,Cail,-68.5,-3.41,-140.62,Independent,1,Anarchy,21-May-2020 00:13


In [77]:
arr = []

for xcs in club_systems_arr:
    cs: FactionInstance = xcs
    arr.append(cs.asArray())

tabulate.tabulate(arr, ['Faction','System','x','y','x','Allegiance','Influence','Vuln.','Updated'],tablefmt='html')

Faction,System,x,y,x,Allegiance,Influence,Vuln.,Updated
Vodyanes Emperor's Grace,27 G. Caeli,75.84,-62.09,-33.34,Empire,17.32,-,20-May-2020 15:51
Worster Insurance,64 Ceti,-37.06,-100.06,-79.12,Independent,2.9,Low Influence,20-May-2020 20:12
Abroin Universal PLC,Abroin,-94.53,110.09,-40.5,Independent,42,-,20-May-2020 02:22
HIP 5183 Emperor's Grace,Achuar,26.09,-231.44,5.94,Empire,12.3,-,20-May-2020 16:55
Rishair Emperor's Grace,Adachit,107.28,-163.84,44.31,Empire,8.31,-,21-May-2020 02:30
Vodyanes Emperor's Grace,Aha Wa,90.38,-56.62,-17.56,Empire,9.99,-,20-May-2020 21:06
Umaspi Emperor's Grace,Ahti,4.06,-79.09,-7.72,Empire,8.5,-,21-May-2020 01:34
Ekono Emperor's Grace,Aiga,66.62,-147.56,102.78,Empire,18.7,-,16-May-2020 08:34
HIP 101031 Emperor's Grace,Ailurii,56.38,-125.28,157.06,Empire,8.8,-,14-Mar-2020 00:50
Hodack Prison Colony,Akandi,62.03,10.75,38.88,Independent,4.7,LowInf+Inf Fail,21-May-2020 00:55


In [0]:
import plotly.express as px

xs = []
ys = []
zs = []
names = []
systemNames = []
allgs = []

for xcs in club_systems_arr:
    cs: FactionInstance = xcs
    #vulnerable = cs.isVulnerable()
    #if (vulnerable == False): continue
    names.append(cs.get_name())
    systemNames.append(cs.getSystemName())
    xs.append(cs.getX())
    ys.append(cs.getY())
    zs.append(cs.getZ())
    allgs.append(cs.get_allegiance())

data = {
  'systemName': systemNames,
  'name': names,
  'x': xs,
  'y': ys,
  'z': zs,
  'allegiance' : allgs
}

df = pd.DataFrame(data=data)


In [83]:
df
fig = px.scatter_3d(df, x='x', y='y', z='z',
              color='allegiance', hover_data=[df.name]) 
#, hover_name=[df.systemName])    hover_name=['systemName'], 
fig.show()

In [86]:
#filter = df[~df['name'].str.contains("Emperor's Grace")]
filter=df
filter

systemName,name,x,y,z,allegiance


In [50]:
!pip install itables


  Created wheel for itables: filename=itables-0.2.1-cp36-none-any.whl size=10701 sha256=331c91e9cb0f49740f87591315a626d6a1832fb7a79f4d063592f17acd75173b
  Stored in directory: /root/.cache/pip/wheels/42/53/2d/4f607a5ca44ee96a6d74f7fab4b4ab73fd4d4b515e65b18f39
Successfully built itables


In [89]:
import itables.interactive
from itables import show
show(df, paging=False)

systemName,name,x,y,z,allegiance


In [92]:
df

systemName,name,x,y,z,allegiance


# All Club Factions as Comma-Delimited Text

In [90]:
for xcs in club_systems_arr:
    cs: FactionInstance = xcs
    #vulnerable = cs.isVulnerable()
    #if (vulnerable == False): continue
    cs.printCSV()

Vodyanes Emperor's Grace,Vodyanes,80.22,-55.66,-14.47,Empire,35.05,-,20-May-2020 20:33
Worster Insurance,64 Ceti,-37.06,-100.06,-79.12,Independent,2.90,Low Influence,20-May-2020 20:12
Abroin Universal PLC,HIP 72726,-103.72,101.66,-38.50,Independent,6.51,-,03-May-2020 04:40
HIP 5183 Emperor's Grace,HIP 5183,33.25,-230.91,4.38,Empire,23.80,-,20-May-2020 16:41
Rishair Emperor's Grace,Rishair,95.72,-160.47,40.47,Empire,6.11,-,20-May-2020 19:43
Umaspi Emperor's Grace,Vivata,17.97,-77.38,-17.59,Empire,32.30,-,20-May-2020 21:29
Ekono Emperor's Grace,Ekono,59.12,-155.16,99.81,Empire,1.60,Low Influence,20-May-2020 22:14
HIP 101031 Emperor's Grace,Teno,57.28,-129.94,153.41,Empire,7.70,-,20-Apr-2020 20:21
Hodack Prison Colony,Vucub Huan,65.31,35.03,52.25,Independent,61.60,-,21-May-2020 01:51
Bill Turner,Alioth,-33.66,72.47,-20.66,Alliance,0.00,-,21-May-2020 02:17
Turner Research Group,Mel 22 Sector GM-V c2-8,-192.12,-194.97,-471.66,Alliance,100.00,-,20-May-2020 13:10
Chere Emperor's Grace,Col 285

# Vulnerable Club Factions as Comma-Delimited Text

In [91]:
for xcs in club_systems_arr:
    cs: FactionInstance = xcs
    vulnerable = cs.isVulnerable()
    if (vulnerable == False): continue
    cs.printCSV()

Worster Insurance,64 Ceti,-37.06,-100.06,-79.12,Independent,2.90,Low Influence,20-May-2020 20:12
Hodack Prison Colony,Akandi,62.03,10.75,38.88,Independent,4.70,LowInf+Inf Fail,21-May-2020 00:55
Trumbo Emperor's Grace,Aowica,116.84,-55.84,84.47,Empire,12.69,Election,20-May-2020 22:30
Aracenus Emperor's Grace,Aracenus,111.31,-65.88,68.00,Empire,8.59,Civil War,20-May-2020 05:15
Asase Ya Emperor's Grace,Asase Ya,96.03,-126.31,21.44,Empire,8.18,Civil War,20-May-2020 14:53
Wo 9812 B Emperor's Grace,Bidjogo,-49.47,-137.00,51.56,Empire,7.60,War,19-May-2020 16:06
NLTT 56379 Emperor's Grace,Boloch,5.88,-105.00,45.50,Empire,8.29,Election,20-May-2020 20:28
Kitche Emperor's Grace,Boloch,5.88,-105.00,45.50,Empire,8.29,Election,20-May-2020 20:28
Botaiti Emperor's Grace,Botaiti,111.03,-143.69,-6.53,Empire,13.19,Election,16-May-2020 22:52
Sirius Hyperspace,Cail,-68.50,-3.41,-140.62,Independent,1.00,Anarchy,21-May-2020 00:13
Candecama Emperor's Grace,Candecama,56.75,-99.19,84.34,Empire,10.61,War,19-May-